<a href="https://colab.research.google.com/github/mjkimcs/portfolio/blob/main/%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%A7%88%EC%9D%B4%EB%8B%9D/24b_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%A7%88%EC%9D%B4%EB%8B%9D_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1에 가까울수록 긍정, 0에 가까울수록 부정  

In [ ]:
!pip install konlpy
!pip install tensorflow

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 13.3MB/s 
     |████████████████████████████████| 460kB 48.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import json
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

okt = Okt() # 형태소 분석기
tokenizer = Tokenizer(19417, oov_token = 'OOV') # 단어집합(빈도수 최소 3이상)의 크기, Out-Of-Vocabulary(단어 집합에 없는 단어)
with open('/content/wordIndex.json') as json_file: # {'영화':4, '좋다':7, ...}
  word_index = json.load(json_file)
  tokenizer.word_index = word_index

loaded_model = load_model('/content/best_model.h5') # 미리 훈련시킨 딥러닝 모델
def sentiment_predict(new_sentence):
    print(new_sentence) # 영화리뷰 문장
    max_len = 30 # 문장길이 분포
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'] # 불용어
    new_sentence = okt.morphs(new_sentence, stem=True) # 형태소 분석기를 사용해 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 빈도수에 따라 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    return score

In [ ]:
from bs4 import BeautifulSoup
import urllib.request as req

page_num = 1
previous_page_result = ""
while True:
    code = req.urlopen("https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=10106&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}".format(page_num))
    soup = BeautifulSoup(code, "html.parser")
    comment = soup.select("li > div.score_reple > p > span")
    if comment[-1].text.strip() == previous_page_result:
        break
    for i in comment:
        i = i.text.strip()
        if i == "관람객":
            continue
        score = sentiment_predict(i)
        if score >= 0.5:
            print("{}% 확률로 긍정".format(score*100))
        else:
            print("{}% 확률로 부정".format((1-score) * 100))
        print("---------------------------")
    previous_page_result = i
    page_num += 1

절대로,그 어떠한 경우에도,칼라로 바꾸지 말야야 할,제일 첫번째 순위 영화
51.8759161233902% 확률로 부정
---------------------------
70년이 넘게 지난 이 영화는 지금봐도 지루하지 않으며 잉그리드버그만의 미모는 상상초월이다
98.72567653656006% 확률로 긍정
---------------------------
멋진 대사들로 가득찬 영화. 이토록 모든 사람들이 사랑할만한 요소들만 고루 모아논 영화가 어디에 있을까.
67.46876239776611% 확률로 긍정
---------------------------
릭의 남자다움과 런드의 애절한 눈빛을 보면서 빠져든 영화 40년대 영화치고 카메라무브먼트와 편집기법이 괜찮았던영화 사랑하는 여인을 떠나보내고 얻는 우정의 소중함
99.49183464050293% 확률로 긍정
---------------------------
빗물의 편지. 대포소리같은 심장박동등.. 등등. 명장면과 명대사가 많은 대단한 영화. 보가트의 카리스마는 대부의 마론 브란도에 필적한 수준이며. 버그만양은 로마의 휴일의 오드리헵번,바람과함께사라지다의 비비안리처럼 존...
87.35770583152771% 확률로 긍정
---------------------------
뭐 하나 빠짐이 없는 영화. 캐스팅 스토리 연출 불후의 음악 보가트의 멋진 트렌치코트 . 심지어 제목도 '카사블랑카'  낭만과 애수의 그 이름
84.80032682418823% 확률로 긍정
---------------------------
정말 멋지다!! 험프리 보가트..그의 냉소적인 분위기와 고독 뒤에는  이루어질수없는 유부녀를  보낼수밖에 없는 마지막 결단.. 진정 남자이다
96.8397855758667% 확률로 긍정
---------------------------
죽기전에 봐라
65.42751491069794% 확률로 부정
---------------------------
이게 40년대 작이라니... 하나도 어색하지 않다 남자배우의 카리스

In [ ]:
from bs4 import BeautifulSoup
import urllib.request as req

page_num = 1
previous_page_result = ""
while True:
    code = req.urlopen("https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=10106&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}".format(page_num))
    soup = BeautifulSoup(code, "html.parser")
    comment = soup.select("li > div.score_reple > p > span")
    if comment[-1].text.strip() == previous_page_result:
        break
    for i in comment:
        i = i.text.strip()
        if i == "관람객":
            continue
        score = sentiment_predict(i)
        if score >= 0.5:
            print("{:.2f}% 확률로 긍정".format(score*100)) # :.2f 소수점 아래 2자리까지만
        else:
            print("{:.2f}% 확률로 부정".format((1-score) * 100))
        print("---------------------------")
    previous_page_result = i
    page_num += 1

절대로,그 어떠한 경우에도,칼라로 바꾸지 말야야 할,제일 첫번째 순위 영화
51.88% 확률로 부정
---------------------------
70년이 넘게 지난 이 영화는 지금봐도 지루하지 않으며 잉그리드버그만의 미모는 상상초월이다
98.73% 확률로 긍정
---------------------------
멋진 대사들로 가득찬 영화. 이토록 모든 사람들이 사랑할만한 요소들만 고루 모아논 영화가 어디에 있을까.
67.47% 확률로 긍정
---------------------------
릭의 남자다움과 런드의 애절한 눈빛을 보면서 빠져든 영화 40년대 영화치고 카메라무브먼트와 편집기법이 괜찮았던영화 사랑하는 여인을 떠나보내고 얻는 우정의 소중함
99.49% 확률로 긍정
---------------------------
빗물의 편지. 대포소리같은 심장박동등.. 등등. 명장면과 명대사가 많은 대단한 영화. 보가트의 카리스마는 대부의 마론 브란도에 필적한 수준이며. 버그만양은 로마의 휴일의 오드리헵번,바람과함께사라지다의 비비안리처럼 존...
87.36% 확률로 긍정
---------------------------
뭐 하나 빠짐이 없는 영화. 캐스팅 스토리 연출 불후의 음악 보가트의 멋진 트렌치코트 . 심지어 제목도 '카사블랑카'  낭만과 애수의 그 이름
84.80% 확률로 긍정
---------------------------
정말 멋지다!! 험프리 보가트..그의 냉소적인 분위기와 고독 뒤에는  이루어질수없는 유부녀를  보낼수밖에 없는 마지막 결단.. 진정 남자이다
96.84% 확률로 긍정
---------------------------
죽기전에 봐라
65.43% 확률로 부정
---------------------------
이게 40년대 작이라니... 하나도 어색하지 않다 남자배우의 카리스마 대사 하나하나가 버릴게 없다
72.29% 확률로 긍정
---------------------------
헐 난 하나도안지루하던데-_- 캐명작 ㄷㄷㄷ 재미없을지알고 미

# 시각화
pyecharts example : https://pyecharts.readthedocs.io/zh/latest/en-us/prepare/  
En us - Charts base - Bar3D 또는 Pie  



In [ ]:
!pip install pyecharts==0.5.10
!pip install pyecharts_snapshot

     |████████████████████████████████| 122kB 5.9MB/s 
     |████████████████████████████████| 522kB 26.3MB/s 
     |████████████████████████████████| 542kB 41.8MB/s 
     |████████████████████████████████| 1.9MB 30.4MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
  Created wheel for jupyter-echarts-pypkg: filename=jupyter_echarts_pypkg-0.1.2-cp36-none-any.whl size=522016 sha256=8678450d6a8b4790310edde8c79638d336bf534428ce44ddbcfae42bdc6e18f2
  Stored in directory: /root/.cache/pip/wheels/62/5c/66/e79276e15e377da9854521c19718ac5a23d827a1b554cf0091
  Created wheel for javascripthon: filename=javascripthon-0.11-cp36-none-any.whl size=526625 sha256=a12d47f4f7756a1b5d886b106fafba2e2c3db666b84e665329bd29661cd3dfc1
  Stored in directory: /root/.cache/pip/wheels/31/18/7e/bd39eb1de4f3128eba5c0d8bfd8bbc2fd07f6c715b6406c30a
  Created wheel for macropy3: filename=macropy3-1.1.0b2-cp36-none-any.whl size=50109 sha256=75e8781289243d70b431a5c0295aa4d372a7f4283c2fd4113649faf8eb1c2f6a
  Sto

In [ ]:
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import json
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

okt = Okt()
tokenizer = Tokenizer(19417, oov_token = 'OOV')
with open('/content/wordIndex.json') as json_file:
  word_index = json.load(json_file)
  tokenizer.word_index = word_index

loaded_model = load_model('/content/best_model.h5')
def sentiment_predict(new_sentence):
    print(new_sentence) # 영화리뷰 문장
    max_len = 30
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'] # 불용어
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if 0.8 <= score <= 1.0:
      return "매우긍정"
    elif 0.6 <= score < 0.8:
      return "긍정"
    elif 0.4 <= score < 0.6:
      return "보통"
    elif 0.2 <= score <0.4:
      return "부정"
    else:
      return "매우부정"


from bs4 import BeautifulSoup
import urllib.request as req

page_num = 1
previous_page_result = ""
result_dic = {"매우긍정":0, "긍정":0, "보통":0, "부정":0, "매우부정":0}
while True:
    code = req.urlopen("https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=10106&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={}".format(page_num))
    soup = BeautifulSoup(code, "html.parser")
    comment = soup.select("li > div.score_reple > p > span")
    if comment[-1].text.strip() == previous_page_result:
        break
    for i in comment:
        i = i.text.strip()
        if i == "관람객":
            continue
        result = sentiment_predict(i) # 매우긍정, 긍정, 보통, 부정, 매우부정 중 하나의 문자열로 반환
        # if score >= 0.5:
        #     print("{:.2f}% 확률로 긍정".format(score*100)) # :.2f 소수점 아래 2자리까지만
        # else:
        #     print("{:.2f}% 확률로 부정".format((1-score) * 100))
        result_dic[result] += 1
        print("---------------------------")
    previous_page_result = i
    page_num += 1


# 감성분석 시각화 - 3차원막대
# grid3D_depth -> grid3d_depth

from pyecharts import Bar3D # pip install pyecharts==0.5.10
# from selenium import webdriver # 브라우저를 자동으로 실행해주는 모듈
# import os

bar3d = Bar3D("감성분석 결과", width=1200, height=600)
x_axis = ["매우긍정", "긍정", "보통", "부정", "매우부정"]
y_axis = []
data = [[0, 0, result_dic["매우긍정"]], [0, 1, result_dic["긍정"]], [0, 2, result_dic["보통"]], [0, 3, result_dic["부정"]], [0, 4, result_dic["매우부정"]]]
range_color = ['#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#ffffbf',
               '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026']
bar3d.add("", x_axis, y_axis, [[d[1], d[0], d[2]] for d in data], 
    is_visualmap=True, visual_range=[0, 170], visual_range_color=range_color, 
    grid3d_width=200, grid3d_depth=40)
bar3d.render("/content/bar3d.html") # 원하는 파일명 지정

# abs_path = os.path.abspath("/content/bar3d.html") # 파일의 전체 경로를 반환
# browser1 = webdriver.Chrome("./chromedriver")
# browser1.get(abs_path)

절대로,그 어떠한 경우에도,칼라로 바꾸지 말야야 할,제일 첫번째 순위 영화
---------------------------
70년이 넘게 지난 이 영화는 지금봐도 지루하지 않으며 잉그리드버그만의 미모는 상상초월이다
---------------------------
멋진 대사들로 가득찬 영화. 이토록 모든 사람들이 사랑할만한 요소들만 고루 모아논 영화가 어디에 있을까.
---------------------------
릭의 남자다움과 런드의 애절한 눈빛을 보면서 빠져든 영화 40년대 영화치고 카메라무브먼트와 편집기법이 괜찮았던영화 사랑하는 여인을 떠나보내고 얻는 우정의 소중함
---------------------------
빗물의 편지. 대포소리같은 심장박동등.. 등등. 명장면과 명대사가 많은 대단한 영화. 보가트의 카리스마는 대부의 마론 브란도에 필적한 수준이며. 버그만양은 로마의 휴일의 오드리헵번,바람과함께사라지다의 비비안리처럼 존...
---------------------------
뭐 하나 빠짐이 없는 영화. 캐스팅 스토리 연출 불후의 음악 보가트의 멋진 트렌치코트 . 심지어 제목도 '카사블랑카'  낭만과 애수의 그 이름
---------------------------
정말 멋지다!! 험프리 보가트..그의 냉소적인 분위기와 고독 뒤에는  이루어질수없는 유부녀를  보낼수밖에 없는 마지막 결단.. 진정 남자이다
---------------------------
죽기전에 봐라
---------------------------
이게 40년대 작이라니... 하나도 어색하지 않다 남자배우의 카리스마 대사 하나하나가 버릴게 없다
---------------------------
헐 난 하나도안지루하던데-_- 캐명작 ㄷㄷㄷ 재미없을지알고 미루다가 이제서야봤네
---------------------------
내가 본 영화 역사상 최고의 결말. 이 말로 밖에 설명할 수 없다.
---------------------------
일사와 릭의 엇갈리는사랑, 진짜 슬프고

In [ ]:
# 감성분석 시각화 - 파이

from pyecharts import Pie

attr = ["매우긍정", "긍정", "보통", "부정", "매우부정"]
v1 = [result_dic["매우긍정"],result_dic["긍정"],result_dic["보통"],result_dic["부정"],result_dic["매우부정"]]
pie = Pie("감정분석 결과")
pie.add("", attr, v1, is_label_show=True)
pie.render("/content/pie.html")

# abs_path = os.path.abspath("/content/pie.html") # 파일의 전체 경로를 반환
# browser2 = webdriver.Chrome("./chromedriver")
# browser2.get(abs_path)